# EvoGFuzz: An Evolutionary Approach to Grammar-Based Fuzzing

**EvoGFuzz** stands for *evolutionary grammar-based fuzzing*. This approach leverages evolutionary optimization techniques to systematically explore the space of a program's potential inputs, with a particular emphasis on identifying inputs that could lead to exceptional behavior. With a user-defined objective, EvoGFuzz can adapt and refine the input generation strategy over time, making it a powerful tool for uncovering software defects and vulnerabilities.

Efficient detection of defects and vulnerabilities hinges on the ability to automatically generate program inputs that are both valid and diverse. One common strategy is to use grammars, which provide structured and syntactically correct inputs. This approach leads to the concept of grammar-based fuzzing, where fuzzing strategies are guided by the rules defined within the grammar.

A further enhancement to this concept is probabilistic grammar-based fuzzing, where competing grammar rules are associated with probabilities that guide their application. By carefully assigning and optimizing these probabilities, we gain considerable control over the nature of the generated inputs. This enables us to direct the fuzzing process towards specific areas of interest—for example, those functions that are deemed critical, have a higher propensity for failures, or have undergone recent modifications. 

In essence, EvoGFuzz represents a potent blend of evolutionary optimization and probabilistic grammar-based fuzzing, poised to reveal hidden defects and vulnerabilities in a targeted and efficient manner.

## Fuzzing a Program

Our program under investigation is `The Calculator`. This program acts as a typical calculator, capable of evaluating not just arithmetic expressions but also trigonometric functions, such as sine, cosine, and tangent. Furthermore, it also supports the calculation of the square root of a given number.

In [1]:
import math

def calculator(inp: str) -> float:
    """
        A simple calculator function that can evaluate arithmetic expressions 
        and perform basic trigonometric functions and square root calculations.
    """
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

**Side Note:** In the `calculator`, we use Python's `eval` function, which takes a string and evaluates it as a Python expression. We provide a dictionary as the second argument to eval, mapping names to corresponding mathematical functions. This enables us to use the function names directly within the input string. 

In [2]:
# Evaluating the cosine of 2π
print(calculator('cos(6*3.141)'))

0.999993677717667


In [3]:
# Calculating the square root of 36
print(calculator('sqrt(6*6)'))

6.0


Each of these calls to the calculator will evaluate the provided string as a mathematical expression, and print the result.

Now, to find new defects, we need to introduce an oracle that tells us if the error that is triggered is something we expect or a new/unkonwn defect. The `OracleResult` is an enum with two possible values, `NO_BUG` and `BUG`. `NO_BUG` donates a passing test case and `BUG` a failing one.

We import the `OracleResult` enumerated type from the `evogfuzz` library. This is used in the oracle function to indicate the outcome of executing the 'calculator' function with a given input.

In [4]:
from evogfuzz.oracle import OracleResult

This is a function called **oracle**, which acts as an intermediary to handle and classify exceptions produced by the calculator function when given a certain input.

In [5]:
# Make sure you use the OracleResult from the evogfuzz library
from evogfuzz.oracle import OracleResult

def oracle(inp: str):
    """
    This function serves as an oracle or intermediary that catches and handles exceptions 
    generated by the 'calculator' function. The oracle function is used in the context of fuzz testing.
    It aims to determine whether an input triggers a bug in the 'calculator' function.

    Args:
        inp (str): The input string to be passed to the 'calculator' function.

    Returns:
        OracleResult: An enumerated type 'OracleResult' indicating the outcome of the function execution.
            - OracleResult.NO_BUG: Returned if the calculator function executes without any exception or only with CalculatorSyntaxError
            - OracleResult.BUG: Returned if the calculator function raises a ValueError exception, indicating a potential bug.
    """
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.BUG
    
    return OracleResult.NO_BUG

This **oracle** function is used in the context of fuzzing to determine the impact of various inputs on the program under test (in our case the _calculator_). When the calculator function behaves as expected (i.e., no exceptions occur), the **oracle** function returns `OracleResult.NO_BUG`. However, when the `calculator` function raises an unexpected exception, the **oracle** interprets this as a potential bug in the `calculator` and returns `OracleResult.BUG`.

We can see this in action by testing a few initial inputs:

In [6]:
initial_inputs = ['sqrt(1)', 'cos(912)', 'tan(4)']

for inp in initial_inputs:
    print(inp.ljust(20), oracle(inp))

sqrt(1)              NO_BUG
cos(912)             NO_BUG
tan(4)               NO_BUG


The following code represents a simple context-free grammar for our calculator function. This grammar encompasses all the potential valid inputs to the calculator, which include mathematical expressions involving square roots, trigonometric functions, and integer and decimal numbers:

In [7]:
from fuzzingbook.Grammars import Grammar, is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
grammar_alhazen: Grammar = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><onenine><maybe_digits><maybe_frac>"],
    "<maybe_minus>": ["", "-"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": [str(num) for num in range(0, 10)],
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<maybe_frac>": ["", ".<digits>"],
}
    
assert is_valid_grammar(grammar_alhazen)

The defined grammar CALCGRAMMAR provides a structured blueprint for creating various inputs for our fuzz testing. Each rule in this grammar reflects a possible valid input that our calculator function can handle. By fuzzing based on this grammar, we can systematically explore the space of valid inputs to the calculator function.

### Leveraging EvoGFuzz to Unearth New Defects

We apply our `EvoGFuzz` class to carry out fuzz testing using evolutionary grammar-based fuzzing. This is aimed at uncovering potential defects in our 'calculator' function.

To initialize our EvoGFuzz instance, we require a grammar (in our case, `CALCGRAMMAR`), an oracle function, an initial set of inputs, a fitness function, and the number of iterations to be performed in the fuzzing process.

Upon creating the `EvoGFuzz` instance, we can execute the fuzzing process. The `fuzz()` method runs the fuzzing iterations, evolving the inputs based on our fitness function, and returns a collection of inputs that lead to exceptions in the 'calculator' function.

In [8]:
from evogfuzz.evogfuzz_class import EvoGFuzz
from evogfuzz.helper import Tournament_Selection_Mode

epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10,
    tournament_selection_mode = Tournament_Selection_Mode.HIERARCHICAL_LEVENSHTEIN
)

Upon creating the `EvoGFuzz` instance, we can execute the fuzzing process. The `.fuzz()` method runs the fuzzing iterations, evolving the inputs based on our fitness function, and returns a collection of inputs that lead to exceptions in the 'calculator' function.

In [9]:
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

6 ['sqrt(294)', 'cos(11)', 'tan(9142)', 'tan(22)', 'sqrt(41491)', 'tan(11)', 'tan(94491)', 'cos(214114)', 'sqrt(4144)', 'sqrt(491421)', 'tan(1222)']
2 ['tan(9)', 'cos(4)', 'cos(1)', 'tan(1)', 'cos(2)', 'tan(2)', 'tan(4)', 'cos(9)']
1 ['tan(12)', 'cos(92)', 'sqrt(9)', 'sqrt(4)', 'cos(21)', 'tan(19)', 'tan(24)', 'sqrt(2)', 'sqrt(1)', 'cos(49)', 'cos(94)', 'tan(41)']
5 ['sqrt(411)', 'sqrt(211)', 'sqrt(224)', 'tan(4142)']
3 ['sqrt(11)', 'cos(944)', 'cos(121)', 'tan(221)', 'sqrt(99)', 'cos(114)']
4 ['tan(214)', 'cos(914)', 'sqrt(41)', 'sqrt(42)']
6 11
2 8
1 12
5 4
3 6
4 4
8 ['cos(1144119)', 'tan(999949.49)', 'sqrt(444)', 'sqrt(119)', 'cos(999)', 'tan(44.99)', 'tan(419.4)', 'tan(9.19)', 'cos(444)', 'sqrt(141)', 'sqrt(1414)', 'tan(91114)', 'cos(11111)', 'sqrt(9141.1)', 'cos(9.94)', 'tan(491.994)', 'sqrt(1141.4)', 'cos(9.141)', 'cos(4144)', 'cos(1911.91449119)', 'cos(9.911111994)', 'cos(419.4)', 'sqrt(499.11)', 'cos(19141199)', 'tan(91911)', 'tan(91191.11)', 'tan(4.9194)', 'sqrt(9494191)', 'co

3 ['sqrt(-6.2)', 'sqrt(-6.8)', 'sqrt(-4.8)', 'sqrt(-4.6)', 'sqrt(-6.4)', 'sqrt(-9.3)', 'sqrt(-8.6)', 'sqrt(-4.7)', 'sqrt(-3.6)', 'sqrt(-6.3)', 'sqrt(-2.4)', 'sqrt(-4.9)', 'sqrt(-2.6)', 'sqrt(-8.4)', 'sqrt(-6.7)', 'sqrt(-2.8)', 'sqrt(-9.4)', 'sqrt(-7.4)']
6 ['sqrt(-585.2)', 'sqrt(-141.8)']
2 ['sqrt(-5.3)', 'sqrt(-3.5)', 'sqrt(-5.6)', 'sqrt(-9.5)', 'sqrt(-8.5)', 'sqrt(-5.8)', 'sqrt(-6.5)', 'sqrt(-4.5)', 'sqrt(-7.5)', 'sqrt(-5.7)', 'sqrt(-5.4)']
5 ['sqrt(-6.6)', 'sqrt(-7.7)', 'sqrt(-5.5)', 'sqrt(-1.1)', 'sqrt(-8.8)']
7 ['sqrt(-4.49)', 'sqrt(-5.11)', 'sqrt(-15.1)', 'sqrt(-11.2)', 'sqrt(-6.61)', 'sqrt(-1.55)']
1 ['sqrt(-8.1)', 'sqrt(-1.9)', 'sqrt(-1.7)', 'sqrt(-5.1)', 'sqrt(-3.1)', 'sqrt(-1.6)', 'sqrt(-4.1)', 'sqrt(-1.5)', 'sqrt(-7.1)', 'sqrt(-1.8)', 'sqrt(-6.1)', 'sqrt(-1.3)', 'sqrt(-1.4)']
4 ['sqrt(-1.95)', 'sqrt(-82.4)', 'sqrt(-1.58)', 'sqrt(-8.63)', 'sqrt(-87.1)', 'sqrt(-5.48)', 'sqrt(-4.71)']
3 18
6 2
2 11
5 5
7 6
1 13
4 7
EvoGFuzz found 126 bug-triggering inputs!


Lastly, we can examine the inputs that resulted in exceptions. This output can provide valuable insight into potential weaknesses in the 'calculator' function that need to be addressed.

In [10]:
# print only the first 20 bug-triggering inputs
for inp in list(found_exception_inputs)[:20]:
    print(str(inp))

sqrt(-9.919)
sqrt(-944)
sqrt(-15.1)
sqrt(-3.6)
sqrt(-7.5)
sqrt(-6.7)
sqrt(-4.2)
sqrt(-6.2)
sqrt(-4.49)
sqrt(-4.6)
sqrt(-99999)
sqrt(-91)
sqrt(-9.7)
sqrt(-9.99)
sqrt(-6.61)
sqrt(-47556.22674836)
sqrt(-4)
sqrt(-6.8)
sqrt(-6.6)
sqrt(-4494)


In [11]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10,
    tournament_selection_mode = Tournament_Selection_Mode.HIERARCHICAL_JARO
)
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

1
3 ['cos(24)', 'cos(14)', 'cos(91)', 'tan(12)', 'cos(12)', 'tan(91)', 'cos(92)', 'sqrt(9)', 'sqrt(4)', 'cos(42)', 'sqrt(2)', 'sqrt(1)']
6 ['cos(11)', 'sqrt(912491)', 'cos(119)', 'sqrt(12911)', 'tan(22)', 'sqrt(11)', 'tan(11)', 'sqrt(921)', 'sqrt(22)']
1 ['tan(9)', 'cos(9)', 'cos(4)', 'cos(1)', 'tan(1)', 'cos(2)', 'tan(2)', 'tan(4)', 'tan(14999)']
2 ['tan(14)', 'tan(41)']
4 ['sqrt(41)', 'cos(291)', 'cos(214)', 'cos(941)', 'sqrt(49)', 'sqrt(42)']
5 ['cos(4111)', 'sqrt(141)', 'tan(9244)', 'cos(111)', 'sqrt(414)', 'sqrt(499)', 'sqrt(111994)']
3 12
6 9
1 9
2 2
4 6
5 7
1
2 ['tan(11.2912294992191112)', 'tan(29.912499922)', 'sqrt(4192.21)', 'sqrt(429.912)', 'sqrt(124219214.9192)', 'tan(2.1212)', 'sqrt(4.1292)', 'tan(919921.22)', 'tan(224419.9)', 'sqrt(29991)']
9 ['tan(9)', 'cos(2.22129)', 'tan(9.2)', 'sqrt(11.99199)', 'tan(9.991)', 'tan(2)', 'sqrt(2222.2)', 'tan(11.1)', 'cos(19.99)', 'cos(12.911191)', 'tan(21.991919)', 'tan(4229.14)', 'tan(2.1)', 'cos(2.22)']
6 ['cos(1912)', 'sqrt(12.29)', 't

In [12]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10,
    tournament_selection_mode = Tournament_Selection_Mode.HIERARCHICAL_FEATURE_COS
)
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

3 ['cos(121112)', 'cos(11)', 'cos(4111)', 'cos(119121)']
4 ['cos(4)', 'cos(99)', 'cos(91)', 'cos(1)', 'cos(911)', 'cos(2)', 'cos(21)', 'cos(194)', 'cos(19)', 'cos(9)']
2 ['tan(914)', 'tan(14)', 'tan(41)', 'tan(1)', 'tan(141)', 'tan(111)', 'tan(419)', 'tan(2419)', 'tan(114)']
1 ['tan(9)', 'tan(2)', 'tan(92)', 'tan(24)', 'tan(4)', 'tan(42)', 'tan(922)']
6 ['sqrt(1211)', 'sqrt(41)', 'sqrt(11)', 'sqrt(91)', 'sqrt(1214)', 'sqrt(1294)', 'sqrt(19)', 'sqrt(1)']
5 ['sqrt(9)', 'sqrt(949)', 'sqrt(4)', 'sqrt(2)', 'sqrt(92)', 'sqrt(49)', 'sqrt(99)']
3 4
4 10
2 9
1 7
6 8
5 7
5 ['sqrt(491)', 'sqrt(419)', 'sqrt(4912)', 'sqrt(9242)', 'sqrt(121)', 'sqrt(924)', 'sqrt(41)', 'sqrt(-91)', 'sqrt(-29)', 'sqrt(19)', 'sqrt(21249192)', 'sqrt(912)', 'sqrt(112)', 'sqrt(21)', 'sqrt(942)', 'sqrt(-9144)', 'sqrt(-41)', 'sqrt(9)', 'sqrt(11)', 'sqrt(91)', 'sqrt(-92)', 'sqrt(-19)', 'sqrt(-2191)', 'sqrt(-1)', 'sqrt(1)']
2 ['tan(9199)', 'tan(-9919)']
3 ['tan(111494)', 'tan(924)', 'tan(14)', 'tan(12)', 'tan(1)', 'tan(91)', 

7 ['sqrt(-86)', 'sqrt(98)', 'sqrt(93.8)', 'sqrt(85)', 'sqrt(-81)', 'sqrt(-8.2)', 'sqrt(-846.4)', 'sqrt(849)', 'sqrt(-89)', 'sqrt(-5.84)', 'sqrt(8)', 'sqrt(-5.98)', 'sqrt(-2.8)', 'sqrt(-428)', 'sqrt(1648)', 'sqrt(83)', 'sqrt(8.1)', 'sqrt(-8)', 'sqrt(-18)', 'sqrt(3.8)', 'sqrt(483)']
5 ['sqrt(-6)', 'sqrt(26)', 'sqrt(-21)', 'sqrt(-32)', 'sqrt(-42)', 'sqrt(41)', 'sqrt(-2)', 'sqrt(4)', 'sqrt(39.6)', 'sqrt(-56)', 'sqrt(-64)', 'sqrt(-24)', 'sqrt(6)', 'sqrt(-4)', 'sqrt(-63)', 'sqrt(-3)', 'sqrt(9)', 'sqrt(14)', 'sqrt(-52)', 'sqrt(-93)', 'sqrt(-9)', 'sqrt(1)']
8 ['sqrt(-85385)', 'sqrt(8.8)', 'sqrt(386.188)', 'sqrt(-8.88)', 'sqrt(-13.842834)', 'sqrt(-818.2)']
4 ['tan(-48.5)', 'tan(-8)', 'tan(-68)', 'tan(248)']
1 ['tan(2)', 'tan(-9)', 'tan(-4)', 'tan(-43)', 'tan(-6)']
6 ['sqrt(-942.43)', 'sqrt(1393)', 'sqrt(22)', 'sqrt(99)', 'sqrt(2.249)', 'sqrt(-9.249)']
2 ['tan(9)', 'tan(9.4)', 'tan(46)', 'tan(54)']
3 ['tan(8.83)', 'tan(688)']
7 21
5 22
8 6
4 4
1 5
6 6
2 4
3 2
EvoGFuzz found 179 bug-triggering in

In [13]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10,
)
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 52 bug-triggering inputs!


This process illustrates the power of evolutionary grammar-based fuzzing in identifying new defects within our system. By applying evolutionary algorithms to our fuzzing strategy, we can guide the search towards more defect-prone regions of the input space.

#### Analyzing and Sorting All Generated Inputs by Fitness

After the fuzzing process, you may want to examine all the generated inputs. These can be accessed using the `get_all_inputs()` method. Additionally, we can sort these inputs based on their fitness scores to gain insights into which inputs performed best according to our fitness function.

In [14]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

Now, let's print out these sorted inputs along with their respective fitness scores. Inputs with higher fitness scores will be displayed first, as these are the ones our evolutionary process deemed more likely to uncover potential defects.

In [15]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-29942)                             fitness: 1
sqrt(-291991)                            fitness: 1
sqrt(-922)                               fitness: 1
sqrt(-12)                                fitness: 1
sqrt(-292)                               fitness: 1
sqrt(-92124)                             fitness: 1
sqrt(-219129)                            fitness: 1
sqrt(-94)                                fitness: 1
sqrt(-299)                               fitness: 1
sqrt(-22222122)                          fitness: 1
sqrt(-24)                                fitness: 1
sqrt(-491)                               fitness: 1
sqrt(-149)                               fitness: 1
sqrt(-94111)                             fitness: 1
sqrt(-29492412129)                       fitness: 1
sqrt(-4)                                 fitness: 1
sqrt(-12924)                             fitness: 1
sqrt(-12249)                             fitness: 1
sqrt(-911)                               fitness: 1
sqrt(-21291)

This output provides an overview of the evolved inputs and their effectiveness in revealing potential defects, as gauged by our fitness function. It is a valuable resource for understanding the behavior of our program under various inputs and the effectiveness of our evolutionary grammar-based fuzzing approach.

### Incorporating Custom Fitness Functions

The fitness function plays a crucial role in guiding the evolution process of our fuzzing inputs. A well-crafted fitness function can effectively direct the search towards the most promising regions of the input space.

To create your own fitness function, define a function that takes an `Input` instance and returns a float value. The return value represents the 'fitness' of the given input, with higher values indicating better fitness. Here is a simple template:

```python
from evogfuzz.input import Input

def fitness_function_XYZ(inp: Input) -> float:
    # Implement your fitness function here.
    return 0.0
```

For instance, suppose we're interested in inputs that invoke the cosine function in our calculator. We could define a fitness function `fitness_function_cos` that assigns a high fitness value to inputs containing 'cos'. (**Note that this might not be the best fitness function to find new expcetions.**)

In [16]:
from difflib import SequenceMatcher
from evogfuzz.input import Input

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def fitness_function_XYZ(inp: Input) -> float:
    return similar("sqrt(-", str(inp)[:6])

In [17]:
from evogfuzz.input import Input

def fitness_function_cos(inp: Input) -> float:
    if 'cos' in str(inp):
        return 1.0
    else:
        return 0.0

Once your fitness function is defined, you can incorporate it into the `EvoGFuzz` instance by passing it as the `fitness_function` argument. 

In [18]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=fitness_function_XYZ,
    iterations=10,
)

found_exception_inputs = epp.fuzz()

print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

for inp in found_exception_inputs:
    print(str(inp))

EvoGFuzz found 293 bug-triggering inputs!
sqrt(-42.4)
sqrt(-99.114)
sqrt(-18.58215)
sqrt(-944)
sqrt(-14.44)
sqrt(-911911)
sqrt(-55.57)
sqrt(-94.49)
sqrt(-85)
sqrt(-419)
sqrt(-4494.9)
sqrt(-11.55)
sqrt(-9911.4194999)
sqrt(-49.44)
sqrt(-2.29114)
sqrt(-4944.94)
sqrt(-4.44)
sqrt(-4.2)
sqrt(-9499.1)
sqrt(-42422.4)
sqrt(-59.95)
sqrt(-5551.1755815)
sqrt(-8171755.1)
sqrt(-9.44)
sqrt(-1.19)
sqrt(-95)
sqrt(-44.949)
sqrt(-91.94491)
sqrt(-91)
sqrt(-2)
sqrt(-29.44)
sqrt(-492411)
sqrt(-4444.44)
sqrt(-55.111534192)
sqrt(-14.4)
sqrt(-444)
sqrt(-11411)
sqrt(-141.11)
sqrt(-4.4)
sqrt(-55.5)
sqrt(-9914)
sqrt(-1.177)
sqrt(-1.5591)
sqrt(-4)
sqrt(-41111)
sqrt(-44441.41)
sqrt(-811.8)
sqrt(-4119)
sqrt(-4942.994914)
sqrt(-44.494)
sqrt(-5595)
sqrt(-9.94)
sqrt(-4491)
sqrt(-49444419)
sqrt(-14199.41)
sqrt(-41.411491)
sqrt(-449.9149)
sqrt(-444.1)
sqrt(-8958.8)
sqrt(-59.1)
sqrt(-441.4)
sqrt(-14.144914449)
sqrt(-9.1)
sqrt(-494.44)
sqrt(-41442.99)
sqrt(-1414)
sqrt(-595.515855)
sqrt(-424.114942)
sqrt(-41441)
sqrt(-94.1)

This way, the evolutionary grammar-based fuzzing process is now guided by your custom fitness function, focusing more on the areas you deem critical.

#### Evaluating Inputs Based on Custom Fitness Function

When utilizing a custom fitness function, such as `fitness_function_cos` in our case, we expect inputs containing 'cos' to achieve the highest fitness scores. This is because our fitness function assigns a score of 1.0 to any input that includes 'cos'.

To confirm this behavior, we retrieve all inputs generated during the fuzzing process using the `get_all_inputs()` method and sort these inputs based on their fitness scores.

In [19]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

Let's display these sorted inputs along with their fitness scores. The inputs that contain 'cos' should appear first, demonstrating their high fitness value.

In [20]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-911911)                            fitness: 1.0
sqrt(-944)                               fitness: 1.0
sqrt(-419)                               fitness: 1.0
sqrt(-4494.9)                            fitness: 1.0
sqrt(-4944.94)                           fitness: 1.0
sqrt(-44.949)                            fitness: 1.0
sqrt(-492411)                            fitness: 1.0
sqrt(-55.111534192)                      fitness: 1.0
sqrt(-14.4)                              fitness: 1.0
sqrt(-811.8)                             fitness: 1.0
sqrt(-44441.41)                          fitness: 1.0
sqrt(-44.494)                            fitness: 1.0
sqrt(-4491)                              fitness: 1.0
sqrt(-441.4)                             fitness: 1.0
sqrt(-595.515855)                        fitness: 1.0
sqrt(-441914.9)                          fitness: 1.0
sqrt(-9.4444491)                         fitness: 1.0
sqrt(-4941.19)                           fitness: 1.0
sqrt(-11.1)                 

The resulting output validates the effectiveness of our custom fitness function. It shows how we can guide the evolutionary grammar-based fuzzing process towards specific regions of the input space, thereby facilitating targeted exploration and bug discovery.